This notebook updates the table 1h_prices_latest_enriched  
1h_prices_latest_enriched stores 1 record per item with the latest known price (and maybe volume?)

In [0]:
import pyspark.sql.functions as sf
from delta.tables import DeltaTable

In [0]:
# Read data from 'runescape.02_silver.1h_prices_cleansed'
df_1hour_cleansed = spark.read.table("runescape.02_silver.1h_prices_cleansed")

In [0]:
# Load item mapping data from the Silver Layer
df_item_mapping = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
# enrich price data with map data
df_1hour_enriched = df_1hour_cleansed.join(df_item_mapping, "id")

In [0]:
# create df with only the latest 1h_prices
# get latest time for each id
df_latest_time = df_1hour_enriched.groupBy("id").max("time").withColumnRenamed("max(time)", "time")
df_latest = df_1hour_enriched.join(df_latest_time, ["id", "time"])

In [0]:
df_latest.display()

In [0]:
%skip
df_test = df_latest.groupBy("time").count()
df_test.display()

In [0]:
%skip
# Upsert data into "runescape.02_silver.1hour_latest_enriched"

from pyspark.sql.functions import expr
(df_latest.mergeInto("runescape.02_silver.1hour_latest_enriched", "id")
    .whenMatched().updateAll()
    #.whenNotMatchedBySource().delete()
    .merge())


In [0]:
# Upsert data into "runescape.02_silver.1hour_latest_enriched"

targetDF = DeltaTable.forName(spark, "runescape.02_silver.1hour_latest_enriched")
dfUpdates = df_latest

(targetDF.alias("t")
  .merge(
    source = dfUpdates.alias("s"),
    condition = "t.id = s.id")
  .whenMatchedUpdateAll()
  .execute())

In [0]:
%skip
df_latest.write.mode("overwrite").saveAsTable("runescape.02_silver.1hour_latest_enriched")